In [1]:
import random
from datetime import datetime, timedelta
import pandas as pd
from prophet import Prophet
from forcasting import forecast_featured_items


this is  to import out dependencies

In [14]:
#DEFINING LISTS

items_clothing = ["barot_saya", "barong_tagalog", "patadyong", "dance_uniform", "headdress_kapasti"]
items_instruments = ["kulintang", "glasses_for_binasuan"]
items_tools = ["bamboo_poles", "coconut_shells", "fan_abaniko", "oil_lamps"]
items_misc = ["mirror_panel", "speaker_system", "props_storage_rack"]

all_items = items_clothing + items_instruments + items_tools + items_misc

DEFINING VARIABLES

In [13]:
def generate_mock_data(start_year=2023, end_year=2024, target_count=700):
    records = []

    # date range across 2 years
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    total_days = (end_date - start_date).days

    while len(records) < target_count:
        # random date
        date_obj = start_date + timedelta(days=random.randint(0, total_days))
        month = date_obj.month

        # Decide item category based on month
        if month in [12, 7]:
            item = random.choice(items_instruments + items_clothing)
        elif month in [1, 6, 10, 5]:
            item = random.choice(items_tools + items_clothing)
        elif month in [3, 4, 8]:  # rare months
            if random.random() < 0.2:  # ~20% chance no borrow
                y = 0
                records.append({"ds": date_obj.strftime("%Y-%m-%d"), "y": y, "item": random.choice(all_items)})
                continue
            else:
                item = random.choice(all_items)
        else:
            item = random.choice(all_items)

        # Borrow count logic
        if item in items_clothing:
            y = random.randint(2, 8)
        elif item in items_instruments:
            y = random.randint(1, 5)
        elif item in items_tools:
            y = random.randint(1, 4)
        else:  # misc
            y = random.randint(1, 2)

        records.append({
            "ds": date_obj.strftime("%Y-%m-%d"),
            "y": y,
            "item": item
        })

    # Convert to DataFrame and sort
    df = pd.DataFrame(records)
    df = df[df["y"] > 0]  # skip 0s
    df["ds"] = pd.to_datetime(df["ds"])
    df = df.sort_values("ds").reset_index(drop=True)
    df["ds"] = df["ds"].dt.strftime("%Y-%m-%d")

    return df


#DEFINING FUNCTION FOR RATATAT

In [17]:
def forecast_featured_items(month_num):

    df = pd.read_csv("dataset.csv")
    df["ds"] = pd.to_datetime(df["ds"])
    predictions = {}

    # Convert month number to int
    target_month_num = int(month_num)

    for item_name, group in df.groupby("item"):
        if len(group) < 5:
            continue

        # Aggregate monthly
        monthly = group.groupby(group["ds"].dt.to_period("M"))["y"].sum().reset_index()
        monthly.rename(columns={"ds": "month"}, inplace=True)
        monthly["ds"] = monthly["month"].dt.to_timestamp()
        monthly = monthly[["ds", "y"]]

        # Train Prophet
        model = Prophet()
        model.fit(monthly)

        # Forecast 12 months ahead (1 year)
        future = model.make_future_dataframe(periods=12, freq="ME")
        forecast = model.predict(future)

        # Pick only rows with the target month number
        forecast["month_num"] = forecast["ds"].dt.month
        target_forecast = forecast[forecast["month_num"] == target_month_num]

        if not target_forecast.empty:
            # Take the closest upcoming month
            next_pred = target_forecast.iloc[0]["yhat"]
            predictions[item_name] = next_pred

    # Sort and pick top 5
    top_items = sorted(predictions, key=predictions.get, reverse=True)[:5]

    # print(f"🌟 Featured items for month {month_num} (Top 5):")
    # for idx, item in enumerate(top_items, start=1):
    #     print(f"{idx}. {item}")

    return top_items

PRINTING BEST TOP 5 ITEMS EVERY MONTH

In [25]:
def forecast_all_months(df):
    df["ds"] = pd.to_datetime(df["ds"])
    all_results = {}

    for month in range(1, 13):  # Loop January–December
        predictions = {}

        for item_name, group in df.groupby("item"):
            if len(group) < 5:
                continue

            # Aggregate monthly
            monthly = group.groupby(group["ds"].dt.to_period("M"))["y"].sum().reset_index()
            monthly.rename(columns={"ds": "month"}, inplace=True)
            monthly["ds"] = monthly["month"].dt.to_timestamp()
            monthly = monthly[["ds", "y"]]

            # Train Prophet
            model = Prophet()
            model.fit(monthly)

            # Forecast next 12 months
            future = model.make_future_dataframe(periods=12, freq="ME")
            forecast = model.predict(future)

            # Filter by month
            forecast["month"] = forecast["ds"].dt.month
            target_forecast = forecast[forecast["month"] == month]

            if not target_forecast.empty:
                avg_pred = target_forecast["yhat"].mean()
                predictions[item_name] = round(avg_pred)

        # Take top 5 items for this month
        top_items = sorted(predictions, key=predictions.get, reverse=True)[:5]
        all_results[month] = top_items

    # Print nicely
    for month, items in all_results.items():
        print(f"\n🌟 Featured items for month {month}:")
        for idx, item in enumerate(items, start=1):
            print(f"  {idx}. {item}")

    return all_results

df = pd.read_csv("dataset.csv")  # or Excel
forecast_all_months(df)


13:43:17 - cmdstanpy - INFO - Chain [1] start processing
13:43:17 - cmdstanpy - INFO - Chain [1] done processing
13:43:17 - cmdstanpy - INFO - Chain [1] start processing
13:43:17 - cmdstanpy - INFO - Chain [1] done processing
13:43:18 - cmdstanpy - INFO - Chain [1] start processing
13:43:18 - cmdstanpy - INFO - Chain [1] done processing
13:43:18 - cmdstanpy - INFO - Chain [1] start processing
13:43:18 - cmdstanpy - INFO - Chain [1] done processing
13:43:18 - cmdstanpy - INFO - Chain [1] start processing
13:43:18 - cmdstanpy - INFO - Chain [1] done processing
13:43:18 - cmdstanpy - INFO - Chain [1] start processing
13:43:18 - cmdstanpy - INFO - Chain [1] done processing
13:43:18 - cmdstanpy - INFO - Chain [1] start processing
13:43:19 - cmdstanpy - INFO - Chain [1] done processing
13:43:19 - cmdstanpy - INFO - Chain [1] start processing
13:43:19 - cmdstanpy - INFO - Chain [1] done processing
13:43:19 - cmdstanpy - INFO - Chain [1] start processing
13:43:19 - cmdstanpy - INFO - Chain [1]


🌟 Featured items for month 1:
  1. headdress_kapasti
  2. barot_saya
  3. patadyong
  4. barong_tagalog
  5. dance_uniform

🌟 Featured items for month 2:
  1. barot_saya
  2. patadyong
  3. barong_tagalog
  4. dance_uniform
  5. headdress_kapasti

🌟 Featured items for month 3:
  1. headdress_kapasti
  2. barot_saya
  3. patadyong
  4. barong_tagalog
  5. dance_uniform

🌟 Featured items for month 4:
  1. barot_saya
  2. headdress_kapasti
  3. patadyong
  4. dance_uniform
  5. barong_tagalog

🌟 Featured items for month 5:
  1. barot_saya
  2. headdress_kapasti
  3. patadyong
  4. barong_tagalog
  5. dance_uniform

🌟 Featured items for month 6:
  1. barot_saya
  2. patadyong
  3. dance_uniform
  4. barong_tagalog
  5. headdress_kapasti

🌟 Featured items for month 7:
  1. barot_saya
  2. headdress_kapasti
  3. patadyong
  4. barong_tagalog
  5. dance_uniform

🌟 Featured items for month 8:
  1. barot_saya
  2. headdress_kapasti
  3. patadyong
  4. barong_tagalog
  5. dance_uniform

🌟 Featu

{1: ['headdress_kapasti',
  'barot_saya',
  'patadyong',
  'barong_tagalog',
  'dance_uniform'],
 2: ['barot_saya',
  'patadyong',
  'barong_tagalog',
  'dance_uniform',
  'headdress_kapasti'],
 3: ['headdress_kapasti',
  'barot_saya',
  'patadyong',
  'barong_tagalog',
  'dance_uniform'],
 4: ['barot_saya',
  'headdress_kapasti',
  'patadyong',
  'dance_uniform',
  'barong_tagalog'],
 5: ['barot_saya',
  'headdress_kapasti',
  'patadyong',
  'barong_tagalog',
  'dance_uniform'],
 6: ['barot_saya',
  'patadyong',
  'dance_uniform',
  'barong_tagalog',
  'headdress_kapasti'],
 7: ['barot_saya',
  'headdress_kapasti',
  'patadyong',
  'barong_tagalog',
  'dance_uniform'],
 8: ['barot_saya',
  'headdress_kapasti',
  'patadyong',
  'barong_tagalog',
  'dance_uniform'],
 9: ['barot_saya',
  'patadyong',
  'headdress_kapasti',
  'barong_tagalog',
  'kulintang'],
 10: ['barot_saya',
  'patadyong',
  'headdress_kapasti',
  'kulintang',
  'barong_tagalog'],
 11: ['barot_saya',
  'patadyong',
  

MAIN FUNCTION

In [26]:
x = "02"

if __name__ == "__main__":
    results = forecast_featured_items(x)
    print(results)


13:44:19 - cmdstanpy - INFO - Chain [1] start processing
13:44:19 - cmdstanpy - INFO - Chain [1] done processing
13:44:19 - cmdstanpy - INFO - Chain [1] start processing
13:44:19 - cmdstanpy - INFO - Chain [1] done processing
13:44:19 - cmdstanpy - INFO - Chain [1] start processing
13:44:20 - cmdstanpy - INFO - Chain [1] done processing
13:44:20 - cmdstanpy - INFO - Chain [1] start processing
13:44:20 - cmdstanpy - INFO - Chain [1] done processing
13:44:20 - cmdstanpy - INFO - Chain [1] start processing
13:44:20 - cmdstanpy - INFO - Chain [1] done processing
13:44:20 - cmdstanpy - INFO - Chain [1] start processing
13:44:20 - cmdstanpy - INFO - Chain [1] done processing
13:44:20 - cmdstanpy - INFO - Chain [1] start processing
13:44:20 - cmdstanpy - INFO - Chain [1] done processing
13:44:21 - cmdstanpy - INFO - Chain [1] start processing
13:44:21 - cmdstanpy - INFO - Chain [1] done processing
13:44:21 - cmdstanpy - INFO - Chain [1] start processing
13:44:21 - cmdstanpy - INFO - Chain [1]

['barong_tagalog', 'dance_uniform', 'patadyong', 'barot_saya', 'headdress_kapasti']
